## Reading NIST PFAS Molecule Data

In [ ]:
import sqlite3
import pandas as pd

# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


# Path to your file
db_path = "/teamspace/studios/this_studio/dimspec_nist_pfas.sqlite"

# Connect to database
conn = sqlite3.connect(db_path)

# List all tables to see what’s inside
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables.head(5)


In [ ]:
table_name = 'ms_data'
rows = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
print(f"Compound {table_name}: length -- {len(rows)}:\n")
rows.head(1)

In [ ]:
import sqlite3
import pandas as pd

def preview_table(table_name):
    print(f"\n=== {table_name} ===")
    try:
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        print(df.columns.tolist())
        #display(df)
        print(len(df))
    except Exception as e:
        print(f"Error reading {table_name}: {e}")

# Example: replace this name to inspect others
preview_table("compounds")
preview_table("peaks")
preview_table("ms_data")


## Converting PFAS NIST data to MassSpecGym df format

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import pandas as pd
import requests

# --- Step 1: Connect to your DIMSpec database ---
db_path = "/teamspace/studios/this_studio/dimspec_nist_pfas.sqlite"

conn = sqlite3.connect(db_path)

# --- Step 2: Load relevant tables ---
compounds = pd.read_sql_query("SELECT * FROM compounds;", conn)
peaks = pd.read_sql_query("SELECT * FROM peaks;", conn)
ms_data = pd.read_sql_query("SELECT * FROM ms_data;", conn)

# --- Step 3: Merge to link compound info with spectra ---
merged = (
    ms_data
    .merge(peaks, left_on="peak_id", right_on="id", how="inner")
    .merge(compounds, left_on="sample_id", right_on="id", how="inner")
)

print(f"merged_df len {len(merged)}")
#print(merged.head(1))

# --- Step 4: Prepare mz and intensity lists ---
# The ms_data table has measured_mz and measured_intensity columns (space separated)
def parse_mz_list(mz_str):
    if not isinstance(mz_str, str):
        return []
    return [float(x) for x in mz_str.replace(",", " ").split() if x.strip()]

def parse_intensity_list(int_str):
    if not isinstance(int_str, str):
        return []
    return [float(x) for x in int_str.replace(",", " ").split() if x.strip()]


name_to_smiles = {}

import requests
from rdkit import Chem

import requests, time, random
from rdkit import Chem

BASE = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

def name_to_smiles_pubchem(name: str,
                           max_retries=5,
                           sleep_base=0.8,
                           sleep_jitter=0.3) -> str | None:
    """Get canonical SMILES from PubChem with polite throttling."""
        # caching
    if name in name_to_smiles:
        return name_to_smiles[name]

    print(f"{name}")

    for attempt in range(1, max_retries + 1):
        try:
            # Step 1: name → CID
            r = requests.get(
                f"{BASE}/compound/name/{requests.utils.quote(name)}/cids/JSON",
                timeout=15
            )
            if r.status_code in (429, 503):  # too many / server busy
                raise requests.exceptions.RequestException("Rate limited")
            r.raise_for_status()

            cids = r.json().get("IdentifierList", {}).get("CID", [])
            if not cids:
                return None
            cid = cids[0]

            # Step 2: CID → canonical SMILES
            r2 = requests.get(
                f"{BASE}/compound/cid/{cid}/property/CanonicalSMILES/JSON",
                timeout=15
            )
            if r2.status_code in (429, 503):
                raise requests.exceptions.RequestException("Rate limited")
            r2.raise_for_status()
            props = r2.json().get("PropertyTable", {}).get("Properties", [])
            print(props)

            if props and "ConnectivitySMILES" in props[0]:
                smiles = props[0]["ConnectivitySMILES"]
                name_to_smiles[name] = smiles
                return smiles
        except Exception as e:
            # exponential backoff with jitter
            wait = (sleep_base ** attempt) + random.uniform(0, sleep_jitter)
            print(f"[Attempt {attempt}] {e} → waiting {wait:.1f}s")
            time.sleep(wait)

    print(f"Failed after {max_retries} retries for {name}")
    name_to_smiles[name] = None
    return None

# --- Step 6: Construct final formatted DataFrame ---
final_df = pd.DataFrame({
    "Unnamed: 0": range(len(merged)),
    "name": merged["name"],
    "identifier": ["PFAS_NIST_ROW_{:05d}".format(i) for i in range(1, len(merged)+1)],
    "mzs": merged["measured_mz"].apply(lambda x: ",".join(x.replace("  ", " ").split())),
    "intensities": merged["measured_intensity"].apply(lambda x: ",".join(x.replace("  ", " ").split())),
    "smiles":  merged['name'].apply(name_to_smiles_pubchem),
    "inchikey": None,
    "formula": merged["formula"],
    "precursor_formula": merged["formula"],
    "parent_mass": merged["precursor_mz"],  # approximate
    "precursor_mz": merged["precursor_mz"],
    "adduct": "[M–H]–",
    "instrument_type": "Orbitrap",
    "collision_energy": 30.0,
    "fold": None,
    "simulation_challenge": "dimspec"
})

# --- Step 7: Save or inspect ---
print(f"\n✅ Total PFAS entries extracted: {len(final_df)}")
conn.close()
final_df.tail(5)

In [35]:
#final_df.to_csv('/teamspace/studios/this_studio/files/PFAS_NIST_new_compounds_DIMSpec_formatted_smiles.tsv', index=False, sep='\t')


In [ ]:
# Drop duplicates
# assuming your DataFrame is called df and column is 'smiles'
num_nulls = final_df['smiles'].isna().sum()
print("Number of null SMILES:", num_nulls)
print("Total len", len(final_df))

In [ ]:
# --- 1. Number of unique compounds ---
num_unique_compounds = final_df["identifier"].nunique()

# --- 2. Number of nulls in mzs ---
num_null_mzs = final_df["mzs"].isna().sum()

# --- 3. Number of nulls in intensities ---
num_null_intensities = final_df["intensities"].isna().sum()

# --- 4. Number of nulls in precursor_mz ---
num_null_precursor_mz = final_df["precursor_mz"].isna().sum()

# --- 5. Print results ---
print(f"Number of unique compounds: {num_unique_compounds}")
print(f"Number of rows with null mzs: {num_null_mzs}")
print(f"Number of rows with null intensities: {num_null_intensities}")
print(f"Number of rows with null precursor_mz: {num_null_precursor_mz}")


## Merge PFAS/NIST-new with existing MSG/NIST dataset

In [ ]:
import pandas as pd
pth = '/teamspace/studios/this_studio/files/PFAS_NIST_new_compounds_DIMSpec_formatted_smiles.tsv'

df = pd.read_csv(pth, sep='\t')
df.head(1)


In [ ]:
import pandas as pd

df1 = pd.read_csv('/teamspace/studios/this_studio/files/merged_massspec_nist20_with_fold.tsv', sep='\t')
df2 = pd.read_csv('/teamspace/studios/this_studio/files/PFAS_NIST_new_compounds_DIMSpec_formatted_smiles.tsv', sep='\t')

In [45]:
df_combined = pd.concat([df1, df2], ignore_index=True)

In [49]:
len(df_combined)

737936

In [50]:
df_combined_new = df_combined[
    df_combined['smiles'].notna() &                      # not NaN
    (df_combined['smiles'].astype(str).str.strip() != '') &  # not empty string
    (df_combined['smiles'].astype(str).str.lower() != 'none')  # not literal "none"
].reset_index(drop=True)

In [52]:
#df_combined_new.to_csv('/teamspace/studios/this_studio/files/merged_massspec_nist20_with_nist_new.tsv', index=False, sep='\t')
